<font size="6"><center>**Group Time Series Cross Validation**</font>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run ../nb_config.py

running notebook configuration


In [3]:
import numpy as np
import pandas as pd


In [6]:
from src.data_api import sources, sinks
from src import utils, tags
from sklearn.model_selection import TimeSeriesSplit
from src import mle 

## Cross Validation

In [5]:
y_example = pd.DataFrame(
    data=[[0, 0, 1.], [0,1, 2.], [0, 2, 3.], [0, 3, 4.], [0, 4, 5.], [0, 5, 5.],
          [1, 0, 1.], [1,1, 2.], [1, 2, 3.], [1, 3, 4.], [1, 4, 5.], [1, 5, 5.],
          [2, 0, 1.], [2,1, 2.], [2, 2, 3.], [2, 3, 4.], [2, 4, 5.], [2, 5, 5.]],
    columns=['stock_id', 'time_id', 'x']
).set_index(['stock_id', 'time_id'])
y_example

x
stock_id time_id       
0        0       1.0000
         1       2.0000
         2       3.0000
         3       4.0000
         4       5.0000
         5       5.0000
1        0       1.0000
         1       2.0000
         2       3.0000
         3       4.0000
         4       5.0000
         5       5.0000
2        0       1.0000
         1       2.0000
         2       3.0000
         3       4.0000
         4       5.0000
         5       5.0000

Regular scikit-learn TimeSeries splitter cannot handle a group variable, check the following indeces (they are totally wrong)

https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py

In [7]:
cvts_skl = TimeSeriesSplit(n_splits=2)

In [9]:
for train_index, test_index in cvts_skl.split(y_example):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [0 1 2 3 4 5] TEST: [ 6  7  8  9 10 11]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11] TEST: [12 13 14 15 16 17]


Try custom TimeSeries splitter that can handle groups

https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243

You will need to provide groups as well as actual data

In [10]:
cvts0 = mle.GroupTimeSeriesSplit(n_splits=2)
groups0 = y_example.index.get_level_values('time_id')

In [11]:
for train_index, test_index in cvts0.split(y_example, groups=groups0):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [0, 1, 6, 7, 12, 13] TEST: [2, 3, 8, 9, 14, 15]
TRAIN: [0, 1, 2, 3, 6, 7, 8, 9, 12, 13, 14, 15] TEST: [4, 5, 10, 11, 16, 17]


Verify that a regular TimeSeriesSplit is performed on each group (ticker)